<a href="https://colab.research.google.com/github/yohanesnuwara/pyresim/blob/master/PyReSim_PVT_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyReSim (Python Reservoir Simulation) PVT GUI

Gas, oil, and water PVT correlations Guided User Interface (GUI)

@author: Yohanes Nuwara<br>
@email: ign.nuwara97@gmail.com

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, ToggleButtons
import ipywidgets as widgets

In [ ]:
!git clone https://github.com/yohanesnuwara/pyresim

import sys
sys.path.append('/content/pyresim/functions')

from pvt_correlation import *

Cloning into 'pyresim'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (226/226), done.
remote: Total 1258 (delta 142), reused 0 (delta 0), pack-reused 1031
Receiving objects: 100% (1258/1258), 713.18 KiB | 967.00 KiB/s, done.
Resolving deltas: 100% (767/767), done.


## Gas

Input to gas PVT are:

* `pressure`: Pressure (psia)
* `temp`: Temperature (°F)
* `sg`: Specific gravity
* `x_h2s`: H2S mole fraction in gas
* `x_co2`: CO2 mole fraction in gas

### PVT Calculator

In [ ]:
"""
INPUT
"""

pressure = 2200 # psi
temp = 80 # Fahrenheit
sg = 0.7 # specific gravity
x_h2s = 0.05 # mole fraction of H2S in gas
x_co2 = 0.16 # mole fraction of CO2 in gas

"""
CALCULATOR
"""

# calculate pseudoproperties using Sutton (1985), Wichert and Aziz (1972)
P_pc, T_pc, P_pr, T_pr = gas_pseudoprops(temp, pressure, sg, x_h2s, x_co2)

# calculate z-factor using Dranchuk-Aboukassem (1975)
pseudo_rho, z_factor = gas_zfactor(T_pr, P_pr)

# calculate density
rhogas = gas_density(temp, pressure, sg, z_factor)

# calculate gas FVF
Bg = gas_fvf(z_factor, temp, pressure)

# calculate isothermal compressibility using Trube (1957) and Mattar (1975)
cgas = gas_compressibility(T_pr, P_pr, pseudo_rho, z_factor, P_pc)

# calculate viscosity using Lee et al (1966)
viscogas = gas_mu(temp, rhogas, sg)

print('=== Gas PVT Correlation Calculator === \n')
print('Your Input:')
print('Pressure: {} psia'.format(pressure))
print('Temperature: {} °F'.format(temp))
print('Specific Gravity: {}'.format(sg))
print('H2S Mole Fraction: {}'.format(x_h2s))
print('CO2 Mole Fraction: {} \n'.format(x_co2))

print('PVT Output:')
print('z-factor (Dranchuk-Aboukassem, 1975): {}'.format(z_factor))
print('Density: {} lb/ft3'.format(rhogas))
print('FVF: {} res ft3/scf'.format(Bg))
print('Isothermal compressibility (Trube, 1957; Mattar, 1975): {} microsip'.format(cgas * 1E+6))
print('Viscosity (Lee et al, 1966): {} cp'.format(viscogas))

=== Gas PVT Correlation Calculator === 

Your Input:
Pressure: 2200 psia
Temperature: 80 °F
Specific Gravity: 0.7
H2S Mole Fraction: 0.05
CO2 Mole Fraction: 0.16 

PVT Output:
z-factor (Dranchuk-Aboukassem, 1975): 0.7838763704841918
Density: 9.826813243351449 lb/ft3
FVF: 0.005437782389502583 res ft3/scf
Isothermal compressibility (Trube, 1957; Mattar, 1975): 435.56613399411094 microsip
Viscosity (Lee et al, 1966): 0.01726394862816872 cp


### Compressibility (z) Factor - interactive

In [ ]:
@interact

def interactive_gas_zfactor(temp=(110, 572), sg=(0.57, 1.4), x_h2s=(0, 0.4), 
                            x_co2=(0, 0.4)):
  
  pressure = np.linspace(50, 5000, 451)

  # calculate pseudoproperties using Sutton (1985), Wichert and Aziz (1972)
  P_pc = []; P_pr = []
  for i in range(len(pressure)):
    P_pc_, T_pc_, P_pr_, T_pr = gas_pseudoprops(temp, pressure[i], sg, x_h2s, x_co2)
    P_pc.append(P_pc_)
    P_pr.append(P_pr_)

  # calculate z-factor using Dranchuk-Aboukassem (1975)
  z_factor = []
  for j in range(len(P_pr)):
    pseudo_rho, z_ = gas_zfactor(T_pr, P_pr[j])
    z_factor.append(z_)
  
  # plot z-factor vs pressure
  plt.plot(pressure, z_factor, color='red')
  plt.xlim(min(pressure), max(pressure))
  plt.ylim(0.4, 1.4)
  plt.title('Plot of Gas z-factor vs Pressure', size=15, pad=10)
  plt.xlabel('Pressure (psig)')
  plt.ylabel('z-factor')

  plt.grid()
  plt.show()

interactive(children=(IntSlider(value=341, description='temp', max=572, min=110), FloatSlider(value=0.98499999…

### Density - interactive

In [ ]:
@interact

def interactive_gas_density(temp=(110, 572), sg=(0.57, 1.4), x_h2s=(0, 0.4), 
                            x_co2=(0, 0.4)):
  
  pressure = np.linspace(50, 5000, 451)

  # calculate pseudoproperties using Sutton (1985), Wichert and Aziz (1972)
  P_pc = []; P_pr = []
  for i in range(len(pressure)):
    P_pc_, T_pc_, P_pr_, T_pr = gas_pseudoprops(temp, pressure[i], sg, x_h2s, x_co2)
    P_pc.append(P_pc_)
    P_pr.append(P_pr_)

  # calculate z-factor using Dranchuk-Aboukassem (1975)
  z_factor = []
  for j in range(len(P_pr)):
    pseudo_rho, z_ = gas_zfactor(T_pr, P_pr[j])
    z_factor.append(z_)    
  
  # calculate density
  rhogas = []
  for k in range(len(z_factor)):
    rhogas_ = gas_density(temp, pressure[k], sg, z_factor[k])
    rhogas.append(rhogas_)
  
  # plot density vs pressure
  plt.plot(pressure, rhogas, color='blue')
  plt.xlim(min(pressure), max(pressure))
  plt.ylim(0, 35)
  plt.title('Plot of Gas Density vs Pressure', size=15, pad=10)
  plt.xlabel('Pressure (psig)')
  plt.ylabel('Density (lbm/ft3)')

  plt.grid()
  plt.show()

### Gas FVF - interactive

In [ ]:
@interact

def interactive_gas_fvf(temp=(110, 572), sg=(0.57, 1.4), x_h2s=(0, 0.4), 
                        x_co2=(0, 0.4)):
  
  pressure = np.linspace(50, 5000, 451)

  # calculate pseudoproperties using Sutton (1985), Wichert and Aziz (1972)
  P_pc = []; P_pr = []
  for i in range(len(pressure)):
    P_pc_, T_pc_, P_pr_, T_pr = gas_pseudoprops(temp, pressure[i], sg, x_h2s, x_co2)
    P_pc.append(P_pc_)
    P_pr.append(P_pr_)

  # calculate z-factor using Dranchuk-Aboukassem (1975)
  z_factor = []
  for j in range(len(P_pr)):
    pseudo_rho, z_ = gas_zfactor(T_pr, P_pr[j])
    z_factor.append(z_)
  
  # calculate gas FVF
  Bg = []
  for k in range(len(pressure)):
    Bg_ = gas_fvf(z_factor[k], temp, pressure[k])
    Bg.append(Bg_)
  
  # plot gas FVF vs pressure
  plt.plot(pressure, Bg, color='purple')
  plt.xlim(min(pressure), max(pressure))
  plt.ylim(0, 0.6)
  plt.title('Plot of Gas FVF vs Pressure', size=15, pad=10)
  plt.xlabel('Pressure (psig)')
  plt.ylabel('Gas FVF (res ft3/scf)')

  plt.grid()
  plt.show()  

### Gas Isothermal Compressibility - interactive

In [ ]:
@interact

def interactive_gas_compressibility(temp=(110, 572), sg=(0.57, 1.4), 
                                    x_h2s=(0, 0.4), x_co2=(0, 0.4)):
  
  pressure = np.linspace(50, 5000, 451)

  # calculate pseudoproperties using Sutton (1985), Wichert and Aziz (1972)
  P_pc = []; P_pr = []
  for i in range(len(pressure)):
    P_pc_, T_pc_, P_pr_, T_pr = gas_pseudoprops(temp, pressure[i], sg, x_h2s, x_co2)
    P_pc.append(P_pc_)
    P_pr.append(P_pr_)

  # calculate pseudo-reduced density and z-factor using Dranchuk-Aboukassem (1975)
  pseudo_rho = []; z_factor = []
  for j in range(len(P_pr)):
    pseudo_rho_, z_ = gas_zfactor(T_pr, P_pr[j])
    pseudo_rho.append(pseudo_rho_)
    z_factor.append(z_)

  # calculate isothermal compressibility using Trube (1957) and Mattar (1975)
  cgas = []
  for k in range(len(P_pr)):
    cgas_ = gas_compressibility(T_pr, P_pr[k], pseudo_rho[k], z_factor[k], P_pc[k])
    cgas.append(cgas_)
  
  # plot compressibility vs pressure
  plt.plot(pressure, (np.array(cgas) * 1E+6), color='blue')
  plt.xlim(min(pressure), max(pressure))
  plt.title('Plot of Gas Compressibility vs Pressure', size=15, pad=10)
  plt.xlabel('Pressure (psig)')
  plt.ylabel('Compressibility (microsip)')

  plt.grid()
  plt.show()

### Gas Viscosity - interactive

In [ ]:
@interact

def interactive_gas_viscosity(temp=(110, 572), sg=(0.57, 1.4), x_h2s=(0, 0.4), 
                              x_co2=(0, 0.4)):
  
  pressure = np.linspace(50, 5000, 451)

  # calculate pseudoproperties using Sutton (1985), Wichert and Aziz (1972)
  P_pc = []; P_pr = []
  for i in range(len(pressure)):
    P_pc_, T_pc_, P_pr_, T_pr = gas_pseudoprops(temp, pressure[i], sg, x_h2s, x_co2)
    P_pc.append(P_pc_)
    P_pr.append(P_pr_)

  # calculate z-factor using Dranchuk-Aboukassem (1975)
  z_factor = []
  for j in range(len(P_pr)):
    pseudo_rho, z_ = gas_zfactor(T_pr, P_pr[j])
    z_factor.append(z_)    
  
  # calculate density
  rhogas = []
  for k in range(len(z_factor)):
    rhogas_ = gas_density(temp, pressure[k], sg, z_factor[k])
    rhogas.append(rhogas_)
  
  # calculate viscosity using Lee et al (1966)
  viscogas = []
  for m in range(len(rhogas)):
    viscogas_ = gas_mu(temp, rhogas[m], sg)
    viscogas.append(viscogas_)
  
  # plot viscosity vs pressure
  plt.plot(pressure, viscogas, color='black')
  plt.xlim(min(pressure), max(pressure))
  plt.ylim(0, 0.09)
  plt.title('Plot of Gas Viscosity vs Pressure', size=15, pad=10)
  plt.xlabel('Pressure (psig)')
  plt.ylabel('Viscosity (cp)')

  plt.grid()
  plt.show()

## Oil

Input to oil PVT are:

* `pressure`: Pressure (psia)
* `temp`: Temperature (°F)
* `sg`: Specific gravity
* `Rsb`: Solution gas-oil ratio at bubble-point pressure (scf/STB) (from PVT data)
* `api`: Oil gravity (API)

### PVT Calculator

In [ ]:
"""
INPUT
"""

pressure = 1900 # psi
temp = 310 # Fahrenheit
sg = 0.77 # specific gravity
Rsb = 800 # solution gas-oil ratio at bubble-point pressure (scf/STB) (from PVT data)
api = 38 # oil gravity (API)

"""
CALCULATOR
"""

# calculate bubble-point pressure using Vasquez and Beggs (1980)
pbubble = oil_pbubble(Rsb, sg, api, temp)

# calculate isothermal compressibility using Vazquez and Beggs (1980); McCain et al (1988)
coil = oil_compressibility(pressure, pbubble, temp, api, Rsb, sg)

# calculate FVF using Vazquez and Beggs (1980); Levitan and Murtha (1999)
Bo = oil_fvf(pbubble, api, Rsb, sg, temp, pressure)

# calculate gas-oil ratio using Vazquez and Beggs (1980)
Rs = gasoilratio(pressure, pbubble, sg, api, temp, Rsb)

# calculate gas-oil ratio using Vazquez and Beggs (1980); Beggs and Robinson (1975)
viscooil = oil_mu(pressure, pbubble, sg, api, temp, Rsb)

print('=== Oil PVT Correlation Calculator === \n')
print('Your Input:')
print('Pressure: {} psia'.format(pressure))
print('Temperature: {} °F'.format(temp))
print('Specific Gravity: {}'.format(sg))
print('Gas-oil ratio @ Bubble-point pressure: {} scf/STB'.format(Rsb))
print('Oil gravity: {} API \n'.format(api))

print('PVT Output:')
print('Bubble-point Pressure (Vasquez and Beggs, 1980): {} psi'.format(pbubble))
print('Gas-oil ratio (Vasquez and Beggs, 1980): {} scf/STB'.format(Rs))
print('FVF (Vazquez and Beggs, 1980; Levitan and Murtha, 1999): {} RB/STB'.format(Bo))
print('Isothermal compressibility (Vasquez and Beggs, 1980): {} microsip'.format(coil * 1E+6))
print('Viscosity (Vazquez and Beggs, 1980; Beggs and Robinson, 1975): {} cp'.format(viscooil))

=== Oil PVT Correlation Calculator === 

Your Input:
Pressure: 1900 psia
Temperature: 310 °F
Specific Gravity: 0.77
Gas-oil ratio @ Bubble-point pressure: 800 scf/STB
Oil gravity: 38 API 

PVT Output:
Bubble-point Pressure (Vasquez and Beggs, 1980): 3828.485471648363 psi
Gas-oil ratio (Vasquez and Beggs, 1980): 348.27111660255684 scf/STB
FVF (Vazquez and Beggs, 1980; Levitan and Murtha, 1999): 1.3041017875410919 RB/STB
Isothermal compressibility (Vasquez and Beggs, 1980): 61.009423965846686 microsip
Viscosity (Vazquez and Beggs, 1980; Beggs and Robinson, 1975): 0.36673292316547934 cp


### Oil Bubble-point Pressure - interactive

In [ ]:
@interact

def interactive_oil_pbubble(Rsb=(20, 2070), sg=(0.56, 1.18), api=(16, 58)):
  
  temp = np.linspace(70, 295, 100)

  # calculate bubble-point pressure using Vasquez and Beggs (1980)
  pbubble = []
  for i in range(len(temp)):
    pb = oil_pbubble(Rsb, sg, api, temp[i])
    pbubble.append(pb)
  
  # plot bubble-point pressure vs temperature
  plt.plot(temp, pbubble, color='blue')
  plt.xlim(min(temp), max(temp))
  plt.ylim(50, 24000)
  plt.title('Plot of Oil Bubble-point Pressure vs Temperature', size=15, pad=10)
  plt.xlabel('Temperature (°F)')
  plt.ylabel('Bubble-point Pressure (psia)')

  plt.grid()
  plt.show()

### Oil Isothermal Compressibility - interactive

In [ ]:
@interact

def interactive_oil_compressibility(Rsb=(20, 2070), sg=(0.56, 1.18), api=(16, 58),
                                    temp=(70, 295)):
  
  pressure = np.linspace(50, 5000, 451)

  # calculate bubble-point pressure using Vasquez and Beggs (1980)
  pbubble = oil_pbubble(Rsb, sg, api, temp)

  # calculate isothermal compressibility using Vazquez and Beggs (1980); McCain et al (1988)
  coil = []
  for i in range(len(pressure)):
    coil_ = oil_compressibility(pressure[i], pbubble, temp, api, Rsb, sg)
    coil.append(coil_)
  
  # plot oil FVF vs pressure 
  plt.plot(pressure, (np.array(coil) * 1E+06), color='green')
  plt.xlim(min(pressure), max(pressure))
  plt.ylim(0, 4000)
  plt.title('Plot of Oil Compressibility vs Pressure', size=15, pad=10)
  plt.xlabel('Pressure (psia)')
  plt.ylabel('Oil compressibility (microsip)')

  plt.grid()
  plt.show()

### Oil FVF - interactive

In [ ]:
@interact

def interactive_oil_fvf(Rsb=(20, 2070), sg=(0.56, 1.18), api=(16, 58),
                            temp=(70, 295)):
  
  pressure = np.linspace(50, 5000, 451)

  # calculate bubble-point pressure using Vasquez and Beggs (1980)
  pbubble = oil_pbubble(Rsb, sg, api, temp)

  # calculate FVF using Vazquez and Beggs (1980); Levitan and Murtha (1999)
  Bo = []
  for i in range(len(pressure)):
    Bo_ = oil_fvf(pbubble, api, Rsb, sg, temp, pressure[i])
    Bo.append(Bo_)
  
  # plot oil FVF vs pressure 
  plt.plot(pressure, Bo, color='red')
  plt.xlim(min(pressure), max(pressure))
  plt.ylim(1, 1.5)
  plt.title('Plot of Oil FVF vs Pressure', size=15, pad=10)
  plt.xlabel('Pressure (psia)')
  plt.ylabel('Oil FVF (RB/STB)')

  plt.grid()
  plt.show()

### Gas-Oil Ratio - interactive

In [ ]:
@interact

def interactive_oil_sgor(Rsb=(20, 2070), sg=(0.56, 1.18), api=(16, 58),
                            temp=(70, 295)):
  
  pressure = np.linspace(50, 5000, 451)

  # calculate bubble-point pressure using Vasquez and Beggs (1980)
  pbubble = oil_pbubble(Rsb, sg, api, temp)

  # calculate gas-oil ratio using Vazquez and Beggs (1980)
  Rs = []
  for i in range(len(pressure)):
    Rs_ = gasoilratio(pressure[i], pbubble, sg, api, temp, Rsb)
    Rs.append(Rs_)
  
  # plot oil FVF vs pressure 
  plt.plot(pressure, Rs, color='purple')
  plt.xlim(min(pressure), max(pressure))
  plt.ylim(0, 2000)
  plt.title('Plot of Gas-Oil Ratio vs Pressure', size=15, pad=10)
  plt.xlabel('Pressure (psia)')
  plt.ylabel('Gas-Oil Ratio (scf/STB)')

  plt.grid()
  plt.show()

interactive(children=(IntSlider(value=1045, description='Rsb', max=2070, min=20), FloatSlider(value=0.87, desc…

### Oil Viscosity - interactive

In [ ]:
@interact

def interactive_oil_visco(Rsb=(20, 2070), sg=(0.56, 1.18), api=(16, 58),
                            temp=(70, 295)):
  
  pressure = np.linspace(50, 5000, 451)

  # calculate bubble-point pressure using Vasquez and Beggs (1980)
  pbubble = oil_pbubble(Rsb, sg, api, temp)

  # calculate gas-oil ratio using Vazquez and Beggs (1980); Beggs and Robinson (1975)
  visco = []
  for i in range(len(pressure)):
    visco_ = oil_mu(pressure[i], pbubble, sg, api, temp, Rsb)
    visco.append(visco_)
  
  # plot oil FVF vs pressure 
  plt.plot(pressure, visco, color='green')
  plt.xlim(min(pressure), max(pressure))
  plt.ylim(0, 10)
  plt.title('Plot of Oil Viscosity vs Pressure', size=15, pad=10)
  plt.xlabel('Pressure (psia)')
  plt.ylabel('Oil Viscosity (cp)')

  plt.grid()
  plt.show()

## Water

Input to water PVT are:

* `pressure`: Pressure (psia)
* `temp`: Temperature (°F)

### PVT Calculator

In [ ]:
"""
INPUT
"""

pressure = 3000 # psi
temp = 190 # Fahrenheit

"""
CALCULATOR
"""

# calculate water FVF using McCain et al (1989)
Bw = waterfvf(temp, pressure)

print('=== Water PVT Correlation Calculator === \n')
print('Your Input:')
print('Pressure: {} psia'.format(pressure))
print('Temperature: {} °F'.format(temp))

print('PVT Output:')
print('FVF (McCain et al, 1989): {} RB/STB'.format(Bw))

=== Water PVT Correlation Calculator === 

Your Input:
Pressure: 3000 psia
Temperature: 190 °F
PVT Output:
FVF (McCain et al, 1989): 1.0305404582765743 RB/STB


### Water FVF - interactive

In [ ]:
@interact

def interactive_water_fvf(temp=(70, 295)):
  
  pressure = np.linspace(50, 5000, 451)

  # calculate bubble-point pressure using Vasquez and Beggs (1980)
  pbubble = oil_pbubble(Rsb, sg, api, temp)

  # calculate water FVF using McCain et al (1989)
  Bw = []
  for i in range(len(pressure)):
    Bw_ = waterfvf(temp, pressure[i])
    Bw.append(Bw_)
  
  # plot oil FVF vs pressure 
  plt.plot(pressure, Bw, color='blue')
  plt.xlim(min(pressure), max(pressure))
  plt.ylim(0.98, 1.08)
  plt.title('Plot of Water FVF vs Pressure', size=15, pad=10)
  plt.xlabel('Pressure (psia)')
  plt.ylabel('Water FVF (RB/STB)')

  plt.grid()
  plt.show()

interactive(children=(IntSlider(value=182, description='temp', max=295, min=70), Output()), _dom_classes=('wid…